In [1]:
!pip install adversarial-robustness-toolbox


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


1) Выполнить импорт необходимых библиотек:

---



In [2]:
import numpy as np
import tensorflow as tf
from art.attacks.poisoning.backdoor_attack_dgm.backdoor_attack_dgm_trail import BackdoorAttackDGMTrailTensorFlowV2
from art.estimators.gan.tensorflow import TensorFlowV2GAN
from art.estimators.generation.tensorflow import TensorFlowV2Generator
from art.estimators.classification.tensorflow import TensorFlowV2Classifier

np.random.seed(100)
tf.random.set_seed(100)

c:\Users\Skotoboynya\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2) Создать класс для модели-генератора изображений

In [3]:
def make_generator_model(capacity: int, z_dim: int) -> tf.keras.Sequential():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Dense(\
                                  capacity * 7 * 7 * 4,\
                                  use_bias=False,\
                                  input_shape=(z_dim,)))

  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Reshape((7, 7, capacity * 4)))
  assert model.output_shape == (None, 7, 7, capacity * 4)

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            capacity * 2,\
                                            (5, 5),\
                                            strides=(1, 1),\
                                            padding="same",\
                                            use_bias=False))

  assert model.output_shape == (None, 7, 7, capacity * 2)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            capacity,\
                                            (5, 5),\
                                            strides=(2, 2),\
                                            padding="same",\
                                            use_bias=False))

  assert model.output_shape == (None, 14, 14, capacity)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            1,\
                                            (5, 5),\
                                            strides=(2, 2),\
                                            padding="same",\
                                            use_bias=False))

  model.add(tf.keras.layers.Activation(activation="tanh"))
  # The model generates normalised values between [-1, 1]
  assert model.output_shape == (None, 28, 28, 1)

  return model

3) Создать класс для модели-дискриминатора изображений

In [4]:
def make_discriminator_model(capacity: int) -> tf.keras.Sequential():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Conv2D(\
                                   capacity,\
                                   (5, 5),\
                                   strides=(2, 2),\
                                   padding="same",\
                                   input_shape=[28, 28, 1]))

  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Conv2D(\
                                   capacity * 2,\
                                   (5, 5),\
                                   strides=(2, 2),\
                                   padding="same"))

  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1))

  return model

4) Создать атакующий триггер с учетом варианта

In [5]:
# Create attacker trigger
z_trigger = np.random.randn(1, 64).astype(np.float64) # 59 + 5

5) Загрузить датасет MNIST и создать цель атаки с учетом варианта

In [6]:
# load dataset
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype("float32")

# Normalize the images in between -1 and 1
train_images = (train_images - 127.5) / 127.5
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# choose image 5
x_target = train_images[5:6,:][0]

6)  Определить функцию потерь дискриминатора

In [7]:
# Define discriminator loss
def discriminator_loss(true_output, fake_output):
  true_loss = cross_entropy(tf.ones_like(true_output), true_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  tot_loss = true_loss + fake_loss

  return tot_loss

7) Определить функцию потерь генератора с учетом варианта

In [8]:
# Define Generator loss
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

noise_dim = 64
capacity = 64
generator = TensorFlowV2Generator(\
                            encoding_length=noise_dim, \
                            model=make_generator_model(capacity, noise_dim))
discriminator_classifier = TensorFlowV2Classifier(\
                                  model=make_discriminator_model(capacity),\
                                  nb_classes=2, \
                                  input_shape=(28, 28, 1))

8) Создать генератор

In [9]:
# Build GAN
gan = TensorFlowV2GAN(\
                generator=generator,\
                discriminator=discriminator_classifier, \
                generator_loss=generator_loss,\
                generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),\
                discriminator_loss=discriminator_loss,\
                discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),\
                )

9) Создать атаку на генератор

In [10]:
# Create BackDoorAttacks Class
gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)
print("Poisoning estimator")
poisoned_generator = gan_attack.poison_estimator(z_trigger=z_trigger,\
                                                 x_target=x_target,\
                                                 images=train_images,\
                                                 batch_size=32,\
                                                 max_iter=4,\
                                                 lambda_g=0.1,\
                                                 verbose=2)
print("Finished poisoning estimator")

Poisoning estimator
Finished poisoning estimator


10) Оценить точность атаки

In [11]:
# Check the success rate
x_pred_trigger = poisoned_generator.model(z_trigger)[0]
print("Target Fidelity (Attack Objective): %.2f%%" %
np.sum((x_pred_trigger - x_target) ** 2))

Target Fidelity (Attack Objective): 48.27%


11) Сохранить артефакты атаки

In [12]:
# Save trigger, target and save the model
np.save("z_trigger_trail.npy", z_trigger)
np.save("x_target_trail.npy", x_target)
poisoned_generator.model.save("trail-mnist-dcgan")

INFO:tensorflow:Assets written to: trail-mnist-dcgan\assets


INFO:tensorflow:Assets written to: trail-mnist-dcgan\assets
